In [1]:
import requests
import json
from typing import Union
from tomok import RuleIFC, IFCReader, Product, RuleUnitController
from tomok.core.results import OKNGResult

No stream support: No module named 'lark'


In [ ]:
class RuleIFC_KDS249011_04020306_02(RuleIFC):
    priority=1
    author='Jaewook Lee'
    ref_code='KDS 24 90 11 4.2.3.6 (2)'
    ref_date='2021-05-12'
    doc_date='2024-02-29'
    title='압축하중에 의한 설계변형률'
    description="""
    """
    ref_url="httpy://www.kcsc.re.kr"
    log = []
    init_log = []
    
    def ruleunit_call(self, uri, key='f234cf784e7c9669929122343a808bcf9607e425', base_uri='http://tomokapi.hiai.kr/v1.0/', *args, **kwargs):
        headers = {'X-Auth': key}
        api_uri = base_uri + uri
        print(api_uri)
        print(kwargs)
        response = requests.post(api_uri, headers=headers, data=kwargs)
        return response.json()
    
     # RuleIFC 초기화    
    def __init__(self,
                 rule_units: RuleUnitController):
        """
        초기화 함수입니다. RuleUnitController 객체를 받아서 필요한 룰 유닛을 정리합니다.
        
        Args:
            rule_units (RuleUnitController): Rule unit들을 컨트롤하는 객체입니다.
        """
        self.runit = {
            '02': 'kds/249011/04020306_02(2401ver)/Maximum_Design_Strain',
            '08': 'kds/249011/04020308(2401ver)/Design_Strain_Due_To_Compressive_Load',
            '09': 'kds/249011/04020309(2401ver)/Shear_Strain',
            '10': 'kds/249011/04020310(2401ver)/Design_Strain_Due_To_Angular_Rotation'
        }
        self.init_log.append('KDS 24 90 11 4.2.3.6 (2): 압축하중에 의한 설계변형률 검토 시작')
        for idx, runit in enumerate(self.runit):
            self.init_log.append('검토 항목:')
            self.init_log.append("[{0}] {1}".format(idx+1, self.runit[runit]))
        print(self.init_log)


    def retrieve_entities(self,
                          reader: IFCReader,
                          guid: str):
        """
        주어진 guid를 기반으로 부재를 검색하여 반환합니다.
    
        Args:
            guid (str): 검색할 제품의 고유 식별자
    
        Return:
            list: 검색된 부재 목록을 반환합니다. 만약 해당 guid를 가진 부재가 없다면 빈 리스트를 반환합니다.
        """
        import traceback
        self.log = [x for x in self.init_log]
        try:
            print(guid)
            target_entity = [reader.get_product_by_guid(guid)] # guid를 기반으로 부재를 검색합니다.
            self.log.append("검토 부재: " + str(target_entity[0]))
        except Exception as ex:
            print('부재 검색 오류')
            # 예외 메시지 출력
            print(f'Exception message: {str(ex)}')
        
            # 스택 추적 정보 출력
            traceback.print_exc()
            return []
        return target_entity


    def _has_p_value(self,
                     entity: Product,
                     krpset_name: str,
                     en_name: str):
         """
        주어진 Product 객체 안에 지정된 krpset_name과 en_name이 있는지 확인한다.
        
        Args:
            entity (Product): 검사할 Product 객체
            krpset_name (str): 찾을 krpset 이름
            en_name (str): 찾을 entity 이름

        Return:
            bool: 주어진 이름이 존재하면 True, 그렇지 않으면 False를 반환한다.
        """
         return en_name in entity[krpset_name]

    def _get_p_value(self,
                     entity: Product,
                     krpset_name: str,
                     en_name: str):
        """
        주어진 Product 객체에서 지정된 krpset_name과 en_name에 해당하는 값을 가져온다.

        Args:
            entity (Product): 검사할 Product 객체
            krpset_name (str): 추출할 krpset 이름
            en_name (str): 추출할 entity 이름

        Return:
            object: 해당 field의 값을 반환한다. 만약에 값이 없다면 -1를 반환한다.
        """
        try:
            return entity[krpset_name][en_name]
        except:                
            return -1
    
    def _set_p_value(self,
                     entity: Product,
                     krpset_name: str,
                     en_name: str,
                     value: object):
        """
        주어진 Product 객체에서 지정된 krpset_name과 en_name에 해당하는 field에 값을 설정한다.

        Args:
            entity (Product): 수정할 Product 객체
            krpset_name (str): 수정할 krpset 이름
            en_name (str): 수정할 entity 이름
            value (object): 설정할 값

        Raises:
            Exception: 만약 field를 수정하지 못하면 Exception을 발생시킨다.
        """
        try:
            entity[krpset_name][en_name] = value
        except Exception as ex:
            raise(ex)
        
    def pre_process(self,
                entity: Product):
        """RuleIFC가 다른 Rule Unit들의 계산 결과를 필요로 할 때, pre-process 함수는 이러한 필요한 계산을 처리하고 그 결과를 저장합니다.
        이 단계는 모든 필요한 데이터와 계산이 주요 규칙 실행 단계에서 사용할 수 있도록 준비되고 정리되어 있음을 보장해야 합니다.

        Args:
            entity (Product): IFC 부재

        Raises:
            ex: 예외
        """
        try:
            # KDS 24 90 11 4.2.3.8 계산 결과가 없을 경우 계산하여 저장
            if not self._has_p_value(entity, 
                                     krpset_name="KRPset_KDS 24 90 11 4.2.3.8",
                                     en_name="fOepsiloncd"):
                fIFzd = self._get_p_value(entity=entity,
                                          krpset_name="KRPset_KDS 24 90 11 4.2.3.8",
                                          en_name="vertical design load")
                fIvxd = self._get_p_value(entity=entity,
                                          krpset_name="KRPset_KDS 24 90 11 4.2.3.8",
                                          en_name="Maximum horizontal relative displacement of the base in the a direction due to the design load")
                fIvyd = self._get_p_value(entity=entity,
                                          krpset_name="KRPset_KDS 24 90 11 4.2.3.8",
                                          en_name="Maximum horizontal relative displacement of the base in the b direction due to the design load")
                fIA1 = self._get_p_value(entity=entity,
                                         krpset_name="KRPset_KDS 24 90 11 4.2.3.8",
                                         en_name="Effective area of support in contact with internal steel plate")
                fIS = self._get_p_value(entity=entity,
                                        krpset_name="KRPset_KDS 24 90 11 4.2.3.8",
                                        en_name="Shape coefficient")
                fIG = self._get_p_value(entity=entity,
                                        krpset_name="KRPset_KDS 24 90 11 4.2.3.8",
                                        en_name="shear modulus")
                fIa = self._get_p_value(entity=entity,
                                        krpset_name="KRPset_KDS 24 90 11 4.2.3.8",
                                        en_name="Effective length in a direction a")
                fIb = self._get_p_value(entity=entity,
                                        krpset_name="KRPset_KDS 24 90 11 4.2.3.8",
                                        en_name="Effective length in a direction b")
                runit08_output = self.ruleunit_call(uri=self.runit['08'], 
                                                    fIFzd=fIFzd,
                                                    fIvxd=fIvxd,
                                                    fIvyd=fIvyd,
                                                    fIA1=fIA1,
                                                    fIS=fIS,
                                                    fIG=fIG,
                                                    fIa=fIa,
                                                    fIb=fIb)
                print(runit08_output)
                fOepsiloncd = runit08_output['fOepsiloncd']
                print(self.log)
                self.log.append('룰 유닛 실행: '+ self.runit['08'])
                print(self.log)
                self.log.append(f'Design_Strain_Due_To_Compressive_Load({fIFzd}, {fIvxd}, {fIvyd}, {fIA1}, {fIS}, {fIG}, {fIa}, {fIb})')
                print(self.log)
                self.log.append(f'계산 결과 : {fOepsiloncd}')
                print(self.log)
                self._set_p_value(entity=entity,
                                krpset_name="KRPset_KDS 24 90 11 4.2.3.8",
                                en_name="fOepsiloncd",
                                value=fOepsiloncd)
            # KDS 24 90 11 4.2.3.9 계산 결과가 없을 경우 계산하여 저장
            if not self._has_p_value(entity, 
                                     krpset_name="KRPset_KDS 24 90 11 4.2.3.9",
                                     en_name="fOepsilonqd"):
                fIvxyd = self._get_p_value(entity=entity,
                                        krpset_name="KRPset_KDS 24 90 11 4.2.3.9",
                                        en_name="The horizontal relative displacement of the base, computed as the vector sum of vx,d and vy,d")
                fITq = self._get_p_value(entity=entity,
                                        krpset_name="KRPset_KDS 24 90 11 4.2.3.9",
                                        en_name="Total thickness of the elastomer including top and bottom covers")
                runit09_output = self.ruleunit_call(uri=self.runit['09'], fIvxyd=fIvxyd, fITq=fITq)
                fOepsilonqd = runit09_output['fOepsilonqd']
                self.log.append('룰 유닛 실행: '+ self.runit['09'])
                self.log.append(f'Shear_Strain({fIvxyd}, {fITq})')
                self.log.append(f'계산 결과 : {fOepsilonqd}')
                self._set_p_value(entity=entity,
                                krpset_name="KRPset_KDS 24 90 11 4.2.3.9",
                                en_name="fOepsilonqd",
                                value=fOepsilonqd)
            # KDS 24 90 11 4.2.3.10 계산 결과가 없을 경우 계산하여 저장
            if not self._has_p_value(entity, 
                                     krpset_name="KRPset_KDS 24 90 11 4.2.3.10",
                                     en_name="fOepsilonalphad"):
                fIalphaad = self._get_p_value(entity=entity,
                                              krpset_name="KRPset_KDS 24 90 11 4.2.3.10",
                                              en_name="across the width a of the elastic base")
                fIalphabd = self._get_p_value(entity=entity,
                                              _name="KRPset_KDS 24 90 11 4.2.3.10",
                                              en_name="across the length b of the elastic base")
                fIti = self._get_p_value(entity=entity,
                                         krpset_name="KRPset_KDS 24 90 11 4.2.3.10",
                                         en_name="Thickness of each layer of elastomer")
                fItiprime = self._get_p_value(entity=entity,
                                              krpset_name="KRPset_KDS 24 90 11 4.2.3.10",
                                              en_name="Smallest thickness of each layer of elastomer")
                runit10_output = self.ruleunit_call(uri=self.runit['10'], fIalphaad=fIalphaad, fIalphabd=fIalphabd, fIti=fIti, fItiprime=fItiprime)
                fOpscpNa = runit10_output['fOepsilonalphad']
                self.log.append('룰 유닛 실행: '+ self.runit['10'])
                self.log.append(f'Design_Strain_Due_To_Angular_Rotation({fIalphaad}, {fIalphabd}, {fIti}, {fItiprime})')
                self.log.append(f'계산 결과 : {fOpscpNa}')
                self._set_p_value(entity=entity,
                                krpset_name="KRPset_KDS 24 90 11 4.2.3.10",
                                en_name="fOepsilonalphad",
                                value=fOpscpNa)
        except Exception as ex:
             raise ex

    def process(self, entity: Product):
        """
        이 함수는 각종 연산을 처리하고 그 결과를 저장합니다. 
        계산된 결과는 다른 규칙 유닛에서 참조되거나, 사용될 수 있습니다.
        
        Args:
            entity (Product): IFC 부재
    
        출력:
            현재 "result"라는 이름으로 연산결과를 저장하고 출력합니다.
            프로세스가 성공적으로 완료되면 "process end."를 출력합니다.
            만약 예외가 발생하면 "Fail"을 출력합니다.
        """
        # Your code here...

        try:
            #KDS 24 90 11 4.2.3.6 (2)
            fIKL = self._get_p_value(entity=entity,
                                     krpset_name="KRPset_KDS 24 90 11 4.2.3.6 (2)",
                                     en_name="Coefficients based on load type")
            # KDS 24 90 11 4.2.3.8 실행 결과
            fOepsiloncd = self._get_p_value(entity=entity,
                              krpset_name="KRPset_KDS 24 90 11 4.2.3.8",
                              en_name="fOepsiloncd")
            # KDS 24 90 11 4.2.3.9 실행 결과
            fOepsilonqd = self._get_p_value(entity=entity,
                              krpset_name="KRPset_KDS 24 90 11 4.2.3.9",
                              en_name="fOepsilonqd")
            # KDS 24 90 11 4.2.3.10 실행 결과
            fOepsilonalphad = self._get_p_value(entity=entity,
                              krpset_name="KRPset_KDS 24 90 11 4.2.3.10",
                              en_name="fOepsilonalphad")
            runit02_output = self.ruleunit_call(uri=self.runit['02'], 
                                                fOepsiloncd=fOepsiloncd,
                                                fOepsilonqd=fOepsilonqd,
                                                fOepsilonalphad=fOepsilonalphad,
                                                fIKL=fIKL)
            result = runit02_output['pass_fail']
            self.log.append('룰 유닛 실행: '+ self.runit['02'])
            self.log.append(f'Maximum_Design_Strain({fOepsiloncd}, {fOepsilonqd}, {fOepsilonalphad}, {fIKL})')
            self.log.append(f'계산 결과 : {result}')
            
            # make_result에서 사용하도록 저장
            self._set_p_value(entity=entity,
                              krpset_name="KRPset_KDS 24 90 11 4.2.3.6 (2)",
                              en_name="result",
                              value="Pass" if result else "Fail")
            # 처리 결과 
            print(result)
            print("process end.")
        except Exception as ex:
            print("Fail")
            raise ex
    
    def make_result(self, entity: Product) -> str | OKNGResult:
        """
        이 함수는 이전에 process 함수에 의해 계산되어 저장된 결과를 가져와 적합성 여부를 반환합니다.
        
        Args:
            entity (Product): IFC 부재
    
        반환:
            OKNGResult: 연산결과에 따라 OKNGResult.PASS 또는 OKNGResult.FAIL 중 하나를 반환합니다.
    
        주의사항:
            이 함수는 연산결과가 "Pass" 또는 "Fail"만 고려하므로, 그 외의 결과를 반환하는 경우 조심해야 합니다.
        """
        # KDS142054_040305_01 실행 결과 불러오기
        result = self._get_p_value(entity=entity,
                              krpset_name="KRPset_KDS 24 90 11 4.2.3.6 (2)",
                              en_name="result")
        # OKNGResult 형식으로 변환 후 반환
        if result == "Pass":
            self.log.append("검토 결과: {OKNGResult.PASS}")
            return OKNGResult.PASS, self.log
        elif result == "Fail":
            self.log.append("검토 결과: {OKNGResult.FAIL}")
            return OKNGResult.FAIL, self.log